# Notebook auxiliar para preprocesado de datos

Al comenzar el desarrollo del proyecto, se hizo la entrega de los datasets a estudiar. Estos datasets fueron entregados como pickles de diccionarios que contienen los datos. Estos diccionarios contienen los datos de cada elemento conectado a la microrred en la forma de diccionarios anidados.

Como librerias como sklearn y seaborn trabajan con Datafames de pandas, se llevó a cabo la actividad de transformar los datasets recibidos a formato tabla, para poder ser cargados como un Dataframe en los Notebooks de experimentos.

## Imports

In [1]:
import pickle
import pandas as pd
import numpy as np

## Variables de control

La siguiente variable de control controlael comportamiento del notebook. A continuación se detalla su utilidad:

* NOTEBOOK_FOLDER_PATH: Esta variable de control es usada para indicar la ubicación del directorio de este notebook, para poder hacer los paths a los distintos paths requeridos para leer o guardar datos desde disco. Se recomienda que esta variable tenga el path relativo del directorio donde se encuentra este notebook respecto al directorio de trabajo del servidor Jupyter. El valor predeterminado asume que el reposiorio de este proyecto ha sido clonado en el directorio de trabajo del servidor Jupyter.

In [2]:
# Este valor debe ser un string
NOTEBOOK_FOLDER_PATH = './proyecto-memoria'

## Funciones

In [3]:
# Cargar pickle
def load_pickle(filename: str) -> dict:
    infile = open(filename, 'rb')
    dic = pickle.load(infile, encoding='latin1')
    infile.close()
    return dic

# Cargar pickle y obtener df
def load_df_from_pickle(filename):
    infile = open(filename, 'rb')
    df =  pickle.load(infile, encoding='latin1')
    infile.close()
    return df

# Esta función toma un dataframe que contiene dataframes en sus celdas, y los transofrma en nuevas columnas
# que contienen en sus celdas los valores que les corresponden
# Por que el nombre squash? Porque transformar dataframes anidados en columnas me da la impresión de aplanar
# o aplastar un montón de algo. Perfectamente podría haber sido "desenredar"
def squash(df):
    df_dict = {}
    for x in range(df.shape[0]):
        serie = df.iloc[x,:]
        for index, value in serie.items():
            df_dict.update({f'{serie.name} {index}': f'{value}'})
            
    return pd.DataFrame.from_dict(df_dict, orient='index').T

Esta es la función encargada de transformar los diccionarios anidados en tablas. Esto se consigue al "aplastar" o "desenredar" los diccionarios anidados en cada Dataframe. Seguir comentarios de siguiente código para una explicación más precisa sobre cómo esto se consigue.

In [4]:
def preprocess(input_dic, output_dic):
    # Armar tablas con los diccionarios
    input_df = pd.DataFrame.from_dict(input_dic, orient='index')
    output_df = pd.DataFrame.from_dict(output_dic, orient='index')

    print(f'Input shape: {input_df.shape}, Output shape: {output_df.shape}')

    # En cada columna inicial, se tranforma el diccionario en cada celda en un dataframe.
    # Esto resulta en dataframes anidados.
    input_df['Load'] = input_df['Load'].apply(lambda x: pd.DataFrame.from_dict(x, orient='index').T)
    input_df['ConventionalGenerator'] = input_df['ConventionalGenerator'].apply(lambda x: pd.DataFrame.from_dict(x, orient='index').T)
    input_df['RenewableGenerator'] = input_df['RenewableGenerator'].apply(lambda x: pd.DataFrame.from_dict(x, orient='index').T)
    input_df['Switch'] = input_df['Switch'].apply(lambda x: pd.DataFrame.from_dict(x, orient='index').T)
    input_df['Line'] = input_df['Line'].apply(lambda x: pd.DataFrame.from_dict(x, orient='index').T)
    input_df['Transformer'] = input_df['Transformer'].apply(lambda x: pd.DataFrame.from_dict(x, orient='index').T)
    input_df['Terminal'] = input_df['Terminal'].apply(lambda x: pd.DataFrame.from_dict(x, orient='index').T)

    # Luego, se "aplasta" los datasets anidados, con el fin de otener múltiples columnas de cada dataframe
    squash_input_df = input_df.copy(deep=True)
    squash_input_df['Load'] = squash_input_df['Load'].apply(lambda x: squash(x))
    squash_input_df['ConventionalGenerator'] = squash_input_df['ConventionalGenerator'].apply(lambda x: squash(x))
    squash_input_df['RenewableGenerator'] = squash_input_df['RenewableGenerator'].apply(lambda x: squash(x))
    squash_input_df['Switch'] = squash_input_df['Switch'].apply(lambda x: squash(x))
    squash_input_df['Line'] = squash_input_df['Line'].apply(lambda x: squash(x))
    squash_input_df['Transformer'] = squash_input_df['Transformer'].apply(lambda x: squash(x))
    squash_input_df['Terminal'] = squash_input_df['Terminal'].apply(lambda x: squash(x))

    # Luego, con el fin de mantener el orden y legibilidad del dataset,
    # se renombran las columnas para incluir el nombre de la columna a la cual originalmente pertenecian
    squash_input_df['Load'] = squash_input_df['Load'].apply(lambda x: x.rename(mapper= lambda y: f'Load {y}', axis=1))
    squash_input_df['ConventionalGenerator'] = squash_input_df['ConventionalGenerator'].apply(lambda x: x.rename(mapper= lambda y: f'ConventionalGenerator {y}', axis=1))
    squash_input_df['RenewableGenerator'] = squash_input_df['RenewableGenerator'].apply(lambda x: x.rename(mapper= lambda y: f'RenewableGenerator {y}', axis=1))
    squash_input_df['Switch'] = squash_input_df['Switch'].apply(lambda x: x.rename(mapper= lambda y: f'Switch {y}', axis=1))
    squash_input_df['Line'] = squash_input_df['Line'].apply(lambda x: x.rename(mapper= lambda y: f'Line {y}', axis=1))
    squash_input_df['Transformer'] = squash_input_df['Transformer'].apply(lambda x: x.rename(mapper= lambda y: f'Transformer {y}', axis=1))
    squash_input_df['Terminal'] = squash_input_df['Terminal'].apply(lambda x: x.rename(mapper= lambda y: f'Terminal {y}', axis=1))

    # Obtener columnas inicialaes
    squash2 = squash_input_df[['Load', 'ConventionalGenerator', 'RenewableGenerator', 'Line', 'Terminal', 'Switch']]

    # El siguiente código corrige los índices del dataset, para poder unirlo con los valores de salida
    total_dict = {}
    for x in range(squash2.shape[0]):
        serie = squash2.iloc[x,:]
        inside_dict = {}
        for index, value in serie.items():
            for y in range(value.shape[0]):
                inside_serie = value.iloc[y,:]
                for in_index, in_value in inside_serie.items():
                    inside_dict.update({f'{in_index}': f'{in_value}'})
        total_dict.update({f'{x}': inside_dict})
                
    squash3 = pd.DataFrame.from_dict(total_dict, orient='index')

    # Procesar columnas de tiempo para obtener una ola columna
    df_time = squash_input_df[['Year', 'Month', 'Day', 'Hour']]
    df_time['Date'] = df_time.apply(lambda row: f"{row['Year']:.0f}-{row['Month']:.0f}-{row['Day']:.0f} {row['Hour']:.0f}:00:00", axis=1)

    # Concatenar fecha y salida correspondiente a cada vector de entrada
    transformed_dataset = pd.concat([squash3.reset_index(), df_time['Date']], axis=1).drop(columns=['index'])
    transformed_dataset = pd.concat([transformed_dataset, output_df], axis=1)

    # Transformar numeros de strings a números
    for col in transformed_dataset.columns:
        transformed_dataset[col] = pd.to_numeric(transformed_dataset[col], errors='ignore')
    
    # Procesamiento terminado
    return transformed_dataset

## Preprocesado

Por cada dataset, obtener pickles almacenados en directorio /data_raw/. Por favor, respetar el nombre de los archivos. Se transforman los datasets, se muestra la cabecera para demostrar y se almacena el dataframe como csv en el directorio /data_process/. Esta actividad debería tomar un tiempo no insignificante. En mi hardware, esto tarda 90 minutos aprox.

In [7]:
datasets_dfs = {}

for dataset in ['Yakutia', 'Malaga', 'Alemana']:
    preprocess_df = preprocess(load_pickle('{}/data_raw/{}/PF_RAW_Input_Dataset.pickle'.format(NOTEBOOK_FOLDER_PATH, dataset)), 
                               load_pickle('{}/data_raw/{}/PF_Output_Dataset.pickle'.format(NOTEBOOK_FOLDER_PATH, dataset)))
    
    display(preprocess_df.head().T)
    print('Dataset {} shape: {}'.format(dataset, preprocess_df.shape))
    preprocess_df.to_csv('{}/data_process/{}.csv'.format(NOTEBOOK_FOLDER_PATH, dataset))

Input shape: (70024, 11), Output shape: (70024, 3)


<ipython-input-4-eaa898354605>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['Date'] = df_time.apply(lambda row: f"{row['Year']:.0f}-{row['Month']:.0f}-{row['Day']:.0f} {row['Hour']:.0f}:00:00", axis=1)


,0,1,2,3,4
Load AC Load pini,0.0251668,0.022514,0.0198611,0.0188,0.019386
Load AC Load terminal,T-1,T-1,T-1,T-1,T-1
Load AC Load voltage,1,1,1,1,1
ConventionalGenerator DG1 pini,0.0139203,0.0123581,0.00887026,0.00331427,0.00330832
ConventionalGenerator DG1 voltage,1,1,1,1,1
ConventionalGenerator DG1 loading,0.0251536,0.0227743,0.0176918,0.0111782,0.0111733
ConventionalGenerator DG1 cst_disp,-0.137586,-0.138311,-0.139932,-0.142513,-0.142516
ConventionalGenerator DG1 terminal,T-0,T-0,T-0,T-0,T-0
ConventionalGenerator DG2 pini,0,0,0,0,0
ConventionalGenerator DG2 voltage,1,1,1,1,1


Dataset Yakutia shape: (70024, 41)
Input shape: (42499, 11), Output shape: (42499, 3)


<ipython-input-4-eaa898354605>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['Date'] = df_time.apply(lambda row: f"{row['Year']:.0f}-{row['Month']:.0f}-{row['Day']:.0f} {row['Hour']:.0f}:00:00", axis=1)


,0,1,2,3,4
Load C10 pini,0.772826,0.489097,0.599374,1.41019,2.0443
Load C10 voltage,0.999337,0.99958,0.999486,0.998791,0.998247
Load C10 terminal,A10,A10,A10,A10,A10
Load C4 pini,0.909207,0.575409,0.705146,1.65905,2.40506
Load C4 voltage,0.9998,0.999874,0.999845,0.999635,0.999471
...,...,...,...,...,...
Terminal A9 pload,0.681906,0.431557,0.52886,1.24428,1.8038
Date,2017-1-1 0:00:00,2017-1-1 1:00:00,2017-1-1 2:00:00,2017-1-1 3:00:00,2017-1-1 4:00:00
losses,0.0001,0,0.0001,0.0004,0.0008
renewable,0,0,0,0,0


Dataset Malaga shape: (42499, 79)
Input shape: (61194, 11), Output shape: (61194, 3)


<ipython-input-4-eaa898354605>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['Date'] = df_time.apply(lambda row: f"{row['Year']:.0f}-{row['Month']:.0f}-{row['Day']:.0f} {row['Hour']:.0f}:00:00", axis=1)


,0,1,2,3,4
Load Farm pini,0,0,0,0,0
Load Farm voltage,1.00049,1.00049,1.00049,1.00049,1.00049
Load Farm terminal,BB Farm,BB Farm,BB Farm,BB Farm,BB Farm
Load Industry pini,0,0,0,0,0
Load Industry voltage,1,1,1,1,1
...,...,...,...,...,...
Switch Breaker/Switch terminalj,BB Residential Complex,BB Residential Complex,BB Residential Complex,BB Residential Complex,BB Residential Complex
Date,2016-1-1 0:00:00,2016-1-1 1:00:00,2016-1-1 2:00:00,2016-1-1 3:00:00,2016-1-1 4:00:00
losses,0,0,0,0,0
renewable,1,1,1,0,0


Dataset Alemana shape: (61194, 68)
